<a href="https://colab.research.google.com/github/elliottower/ocp/blob/696DS-Spring21/docs/source/tutorials/colab_train_s2ef_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SchNet S2EF training example

The purpose of this notebook is to demonstrate some of the basics of the Open Catalyst Project's (OCP) codebase and data. In this example, we will train a schnet model for predicting the energy and forces of a given structure (S2EF task). First, ensure you have installed the OCP ocp repo and all the dependencies according to the [README](https://github.com/Open-Catalyst-Project/ocp/blob/master/README.md).

Disclaimer: This notebook is for tutorial purposes, it is unlikely it will be practical to train baseline models on our larger datasets using this format. As a next step, we recommend trying the command line examples. 

## Environment Setup

In [ ]:
# Clone OCP repo and install PyTorch
!git clone https://github.com/Open-Catalyst-Project/ocp.git 
!pip3 install -q torch==1.6.0 torchvision==0.7.0 -f https://download.pytorch.org/whl/cu101/torch_stable.html

Cloning into 'ocp'...
remote: Enumerating objects: 89, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 5118 (delta 60), reused 4 (delta 1), pack-reused 5029
Receiving objects: 100% (5118/5118), 98.82 MiB | 35.59 MiB/s, done.
Resolving deltas: 100% (3495/3495), done.
Looking in links: https://download.pytorch.org/whl/cu101/torch_stable.html
     |████████████████████████████████| 708.0MB 26kB/s 
     |████████████████████████████████| 5.9MB 5.0MB/s 
  Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101
  Found existing installation: torchvision 0.8.1+cu101
    Uninstalling torchvision-0.8.1+cu101:
      Successfully uninstalled torchvision-0.8.1+cu101


In [2]:
# Install OCP dependencies
!pip3 install -q -r ./ocp/docs/source/tutorials/colab_requirements.txt

     |████████████████████████████████| 1.3MB 19.1MB/s 
     |████████████████████████████████| 71kB 11.7MB/s 
     |████████████████████████████████| 61kB 9.9MB/s 
     |████████████████████████████████| 2.1MB 55.6MB/s 
     |████████████████████████████████| 122kB 50.8MB/s 
     |████████████████████████████████| 1.1MB 47.4MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 419kB 50.0MB/s 
     |████████████████████████████████| 163kB 55.0MB/s 
     |████████████████████████████████| 317kB 49.3MB/s 
     |████████████████████████████████| 204kB 58.6MB/s 
     |████████████████████████████████| 3.2MB 54.4MB/s 
     |████████████████████████████████| 133kB 56.4MB/s 
     |████████████████████████████████| 337kB 54.4MB/s 
     |████████████████████████████████| 71kB 10.3MB/s 
     |████████████████████████████████| 829kB 46.9MB/s 
     |████████████████████████████████

In [3]:
# Install OCP module after we have installed other dependencies 
!python3 -m pip install -q --upgrade git+https://github.com/Open-Catalyst-Project/ocp.git

  Cloning https://github.com/Open-Catalyst-Project/ocp.git to /tmp/pip-req-build-6wrc4b2u
  Running command git clone -q https://github.com/Open-Catalyst-Project/ocp.git /tmp/pip-req-build-6wrc4b2u
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


## Imports

In [4]:
import torch
from ocpmodels import models
from ocpmodels.trainers import ForcesTrainer

In [ ]:
# a simple sanity check that a GPU is available
if torch.cuda.is_available():
    print("True")
else:
    print("False")

True


## The essential steps for training an OCP model

1) Download data

2) Preprocess data (if necessary)

3) Define or load a configuration (config), which includes the following
   
   - task
   - model
   - optimizer
   - dataset
   - trainer

4) Train

5) Depending on the model/task there might be intermediate relaxation step

6) Predict

## Download and preprocess data

In [10]:
!python ./ocp/scripts/download_data.py --task s2ef --split "200k" --get-edges --num-workers 1 --ref-energy --path "./ocp/"

--2021-02-23 22:12:55--  https://dl.fbaipublicfiles.com/opencatalystproject/data/s2ef_train_200K.tar
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.74.142, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 359720960 (343M) [application/x-tar]
Saving to: ‘s2ef_train_200K.tar’

s2ef_train_200K.tar 100%[===================>] 343.06M  24.8MB/s    in 14s     

2021-02-23 22:13:09 (24.6 MB/s) - ‘s2ef_train_200K.tar’ saved [359720960/359720960]

Extracting contents...
Uncompressing s2ef_train_200K/s2ef_train_200K: 100% 80/80 [00:39<00:00,  2.05it/s]
Preprocessing data into LMDBs: 100% 199999/200000 [49:05<00:00, 80.66it/s]

Please download the train S2EF 200K dataset from [here](https://github.com/Open-Catalyst-Project/ocp/blob/master/README.md#download-the-datasets). This example assumes the --get-edges flag to be present at preprocessing time.

In [18]:
# set the path to your local lmdb directory
train_src = "./ocp/data/s2ef/200k/train"

## Define config

For this example, we will explicitly define the config; however, a set of default config files exists in the config folder of this repository. Default config yaml files can easily be loaded with the `build_config` util (found in `ocp/ocpmodels/common/utils.py`). Loading a yaml config is preferrable when launching jobs from the command line. We have included our best models' config files [here](https://github.com/Open-Catalyst-Project/ocp/tree/master/configs/s2ef).

**Task** 

In [21]:
task = {
    'dataset': 'trajectory_lmdb', # dataset used for the S2EF task
    'description': 'Regressing to energies and forces for DFT trajectories from OCP',
    'type': 'regression',
    'metric': 'mae',
    'labels': ['potential energy'],
    'grad_input': 'atomic forces',
    'train_on_free_atoms': True,
    'eval_on_free_atoms': True
}

**Model** - SchNet for this example

In [22]:
model = {
    'name': 'schnet',
    'hidden_channels': 1024, # if training is too slow for example purposes reduce the number of hidden channels
    'num_filters': 256,
    'num_interactions': 3,
    'num_gaussians': 200,
    'cutoff': 6.0
}

**Optimizer**

In [23]:
optimizer = {
    'batch_size': 16, # if hitting GPU memory issues, lower this
    'eval_batch_size': 8,
    'num_workers': 8,
    'lr_initial': 0.0001,
    'lr_gamma': 0.1,
    'lr_milestones': [15, 20],
    'warmup_epochs': 10,
    'warmup_factor': 0.2,
    'max_epochs': 1, # used for demonstration purposes
    'force_coefficient': 100,
}

**Dataset**

For simplicity, `train_src` is used for all the train/val/test sets. Feel free to update with the actual S2EF val and test sets, but it does require additional downloads and preprocessing. If you desire to normalize your targets, `normalize_labels` must be set to `True` and corresponding `mean` and `stds` need to be specified. These values have been precomputed for you and can be found in any of the [`base.yml`](https://github.com/Open-Catalyst-Project/ocp/blob/master/configs/s2ef/20M/base.yml#L5-L9) config files.

In [24]:
dataset = [
{'src': train_src, 'normalize_labels': False}, # train set 
{'src': train_src}, # val set (optional)
{'src': train_src} # test set (optional - writes predictions to disk)
]

**Trainer**

Use the `ForcesTrainer` for the S2EF and IS2RS tasks, and the `EnergyTrainer` for the IS2RE task 

In [25]:
trainer = ForcesTrainer(
    task=task,
    model=model,
    dataset=dataset,
    optimizer=optimizer,
    identifier="SchNet-example",
    run_dir="./", # directory to save results if is_debug=False. Prediction files are saved here so be careful not to override!
    is_debug=False, # if True, do not save checkpoint, logs, or results
    is_vis=False,
    print_every=10,
    seed=0, # random seed to use
    logger="tensorboard", # logger of choice (tensorboard and wandb supported)
    local_rank=0,
    amp=False, # use PyTorch Automatic Mixed Precision (faster training and less memory usage)
)

amp: false
cmd:
  checkpoint_dir: ./checkpoints/2021-02-23-23-08-48-SchNet-example
  commit: null
  identifier: SchNet-example
  logs_dir: ./logs/tensorboard/2021-02-23-23-08-48-SchNet-example
  print_every: 10
  results_dir: ./results/2021-02-23-23-08-48-SchNet-example
  seed: 0
  timestamp: 2021-02-23-23-08-48-SchNet-example
dataset:
  normalize_labels: false
  src: ./ocp/data/s2ef/200k/train
gpus: 1
logger: tensorboard
model: schnet
model_attributes:
  cutoff: 6.0
  hidden_channels: 1024
  num_filters: 256
  num_gaussians: 200
  num_interactions: 3
optim:
  batch_size: 16
  eval_batch_size: 8
  force_coefficient: 100
  lr_gamma: 0.1
  lr_initial: 0.0001
  lr_milestones:
  - 15
  - 20
  max_epochs: 1
  num_workers: 8
  warmup_epochs: 10
  warmup_factor: 0.2
task:
  dataset: trajectory_lmdb
  description: Regressing to energies and forces for DFT trajectories from OCP
  eval_on_free_atoms: true
  grad_input: atomic forces
  labels:
  - potential energy
  metric: mae
  train_on_free_at

## Check the model

In [26]:
print(trainer.model)

OCPDataParallel(
  (module): SchNet(hidden_channels=1024, num_filters=256, num_interactions=3, num_gaussians=200, cutoff=6.0)
)


## Train

In [27]:
trainer.train()

forcesx_mae: 0.5628, forcesy_mae: 0.8631, forcesz_mae: 0.7528, forces_mae: 0.7263, forces_cos: -0.0338, forces_magnitude: 1.2994, energy_mae: 40.6625, energy_force_within_threshold: 0.0000, loss: 113.2887, epoch: 0.0001
forcesx_mae: 0.5716, forcesy_mae: 0.6950, forcesz_mae: 0.6544, forces_mae: 0.6403, forces_cos: 0.0035, forces_magnitude: 1.1676, energy_mae: 33.8022, energy_force_within_threshold: 0.0000, loss: 97.5338, epoch: 0.0009
forcesx_mae: 0.4673, forcesy_mae: 0.5315, forcesz_mae: 0.5705, forces_mae: 0.5231, forces_cos: 0.0170, forces_magnitude: 0.9145, energy_mae: 20.9801, energy_force_within_threshold: 0.0000, loss: 73.2487, epoch: 0.0017
forcesx_mae: 0.3805, forcesy_mae: 0.4511, forcesz_mae: 0.4510, forces_mae: 0.4276, forces_cos: 0.0368, forces_magnitude: 0.7372, energy_mae: 17.3934, energy_force_within_threshold: 0.0000, loss: 60.2195, epoch: 0.0025
forcesx_mae: 0.3313, forcesy_mae: 0.3763, forcesz_mae: 0.3744, forces_mae: 0.3607, forces_cos: 0.0263, forces_magnitude: 0.618

device 0: 100%|██████████| 25000/25000 [38:31<00:00, 10.81it/s]
/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


forcesx_mae: 0.0694, forcesy_mae: 0.0863, forcesz_mae: 0.0919, forces_mae: 0.0825, forces_cos: 0.0763, forces_magnitude: 0.1327, energy_mae: 3.3473, energy_force_within_threshold: 0.0000, loss: 11.6600, epoch: 1.0000
### Predicting on test.


device 0: 100%|██████████| 25000/25000 [37:47<00:00, 11.03it/s]
/usr/local/lib/python3.7/dist-packages/ocpmodels/trainers/forces_trainer.py:337: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  predictions["forces"] = np.array(predictions["forces"])
/usr/local/lib/python3.7/dist-packages/ocpmodels/trainers/base_trainer.py:770: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  np.array(gather_results[k])[idx]


Writing results to ./results/2021-02-23-23-08-48-SchNet-example/s2ef_predictions.npz


### Load Checkpoint
Once training has completed a `Trainer` class, by default, is loaded with the best checkpoint as determined by training or validation (if available) metrics. To load a `Trainer` class directly with a pretrained model, specify the `checkpoint_path` as defined by your previously trained model (`checkpoint_dir` in cell 9):

In [28]:
model = {
    'name': 'schnet',
    'hidden_channels': 1024, # if training is too slow for example purposes reduce the number of hidden channels
    'num_filters': 256,
    'num_interactions': 3,
    'num_gaussians': 200,
    'cutoff': 6.0
}

pretrained_trainer = ForcesTrainer(
    task=task,
    model=model,
    dataset=dataset,
    optimizer=optimizer,
    identifier="SchNet-example",
    run_dir="./", # directory to save results if is_debug=False. Prediction files are saved here so be careful not to override!
    is_debug=False, # if True, do not save checkpoint, logs, or results
    is_vis=False,
    print_every=10,
    seed=0, # random seed to use
    logger="tensorboard", # logger of choice (tensorboard and wandb supported)
    local_rank=0,
    amp=False, # use PyTorch Automatic Mixed Precision (faster training and less memory usage)
)

pretrained_trainer.load_pretrained(checkpoint_path="checkpoints/2021-01-06-16-49-04-SchNet-example/checkpoint.pt")

amp: false
cmd:
  checkpoint_dir: ./checkpoints/2021-02-24-01-12-32-SchNet-example
  commit: null
  identifier: SchNet-example
  logs_dir: ./logs/tensorboard/2021-02-24-01-12-32-SchNet-example
  print_every: 10
  results_dir: ./results/2021-02-24-01-12-32-SchNet-example
  seed: 0
  timestamp: 2021-02-24-01-12-32-SchNet-example
dataset:
  normalize_labels: false
  src: ./ocp/data/s2ef/200k/train
gpus: 1
logger: tensorboard
model: schnet
model_attributes:
  cutoff: 6.0
  hidden_channels: 1024
  num_filters: 256
  num_gaussians: 200
  num_interactions: 3
optim:
  batch_size: 16
  eval_batch_size: 8
  force_coefficient: 100
  lr_gamma: 0.1
  lr_initial: 0.0001
  lr_milestones:
  - 15
  - 20
  max_epochs: 1
  num_workers: 8
  warmup_epochs: 10
  warmup_factor: 0.2
task:
  dataset: trajectory_lmdb
  description: Regressing to energies and forces for DFT trajectories from OCP
  eval_on_free_atoms: true
  grad_input: atomic forces
  labels:
  - potential energy
  metric: mae
  train_on_free_at

False

## Predict

If a test has been provided in your config, predictions are generated and written to disk automatically upon training completion. Otherwise, to make predictions on unseen data a `torch.utils.data` DataLoader object must be constructed. Here we reference our test set to make predictions on. Predictions are saved in `{results_file}.npz` in your `results_dir`.

In [29]:
# make predictions on the existing test_loader
predictions = pretrained_trainer.predict(pretrained_trainer.test_loader, results_file="s2ef_results", disable_tqdm=False)

### Predicting on test.


device 0: 100%|██████████| 25000/25000 [36:46<00:00, 11.33it/s]
/usr/local/lib/python3.7/dist-packages/ocpmodels/trainers/forces_trainer.py:337: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  predictions["forces"] = np.array(predictions["forces"])
/usr/local/lib/python3.7/dist-packages/ocpmodels/trainers/base_trainer.py:770: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  np.array(gather_results[k])[idx]


Writing results to ./results/2021-02-24-01-12-32-SchNet-example/s2ef_s2ef_results.npz


In [30]:
energies = predictions["energy"]
forces = predictions["forces"]